<a href="https://colab.research.google.com/github/sindla97/Machine-Learning/blob/main/humana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [7]:
df=pd.read_csv("2022_Competition_Training.csv", low_memory=False)

In [8]:
df.head()

,id,rx_hum_19_pmpm_cost,cmsd2_can_mal_male_genital_pmpm_ct,cmsd2_inj_surgical_pmpm_ct,rev_pm_capd_pmpm_cd_ct,cci_chf_pmpm_ct,rx_phar_cat_safeway_pmpm_ct,cmsd2_ext_car_pmpm_ct,cmsd2_dig_liver_pmpm_ct,cmsd2_mus_arthropath_pmpm_ct,...,cmsd2_ext_nature_pmpm_ct,cmsd2_gus_m_genital_pmpm_ct,atlas_totalpopacs,cmsd2_neo_neo_other_pmpm_ct,cmsd2_dig_hernia_pmpm_ct,cmsd2_sns_blood_pmpm_ct,cmsd2_ext_compl_medical_care_pmpm_ct,rwjf_food_env_inx,atlas_snapspth16,hi_flag
0,100039218.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.08,33511.0,0.0,0.0,0.00,0.0,9.1,NaN,0
1,100042427.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,176797.0,0.0,0.0,0.00,0.0,6.8,0.670459,0
2,100055103.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,380673.0,0.0,0.0,0.00,0.0,8.3,0.879186,0
3,100074556.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,267696.0,0.0,0.0,0.00,0.0,7.3,0.746201,0
4,100079932.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,19095.0,0.0,0.0,0.25,0.0,6.9,NaN,0


In [9]:
df.shape

(48300, 881)

In [25]:
df_null=pd.DataFrame(df.isnull().sum()).iloc[[x for x in range(df.shape[1]) if df.isnull().sum()[x] > 0.50*(df.shape[0])],:]


<ipython-input-25-b90e7bb9022c>:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_null=pd.DataFrame(df.isnull().sum()).iloc[[x for x in range(df.shape[1]) if df.isnull().sum()[x] > 0.50*(df.shape[0])],:]


In [26]:
df_null

,0
credit_bal_nonmtgcredit_60dpd,45173
credit_hh_autobank,45181
credit_num_1stmtg_collections,43469
credit_bal_consumerfinance_new,45188
cms_tot_partd_payment_amt,45981
cms_risk_adj_payment_rate_b_amt,48276
credit_bal_bankcard_severederog,45212
credit_hh_agencyfirstmtg_new,45299
credit_prcnt_mtgcredit,45217
credit_bal_autobank_new,45243


In [28]:
df['hi_flag'].value_counts()

,count
hi_flag,
0,46182
1,2118


In [ ]:
from sklearn.model_selection import *
